# Gerekli kütüphaneleri yükleyin

In [15]:
import pandas as pd

# Veriyi yükler

In [16]:
def load_data(file_path):
    """
    Veriyi yükler.
    """
    df = pd.read_csv(file_path)
    return df

# Veriyi inceler ve özet istatistiklerini döndürür

In [17]:
def inspect_data(df):
    """
    Veriyi inceler ve özet istatistikleri yazdırır.
    """
    print("İlk 10 gözlem:")
    print(df.head(10))
    print("\nDeğişken isimleri:")
    print(df.columns)
    print("\nBetimsel istatistik:")
    print(df.describe().T)
    print("\nBoş değer sayısı:")
    print(df.isnull().sum())
    print("\nDeğişken tipleri:")
    print(df.dtypes)

# total_order ve total_value değişkenlerini oluşturur

In [18]:
def create_new_variables(df):
    """
    Yeni değişkenler oluşturur.
    """
    df["total_order"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
    df["total_value"] = df["customer_value_total_ever_online"] + df["customer_value_total_ever_offline"]
    return df

# Tarih sütunlarını datetime'a çevirir

In [19]:
def convert_date_columns(df):
    """
    Tarih sütunlarını datetime formatına çevirir.
    """
    date_columns = ["first_order_date", "last_order_date", "last_order_date_online", "last_order_date_offline"]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col])
    return df

# Alışveriş kanallarını analiz eder.

In [20]:
def analyze_channels(df):
    """
    Alışveriş kanallarını analiz eder.
    """
    print("\nAlışveriş kanallarındaki müşteri sayıları:")
    print(df["order_channel"].value_counts())
    print("\nToplam alınan ürün sayısı (online):")
    print(df["order_num_total_ever_online"].sum())
    print("\nToplam alınan ürün sayısı (offline):")
    print(df["order_num_total_ever_offline"].sum())
    print("\nToplam harcamalar (online):")
    print(df["customer_value_total_ever_online"].sum())
    print("\nToplam harcamalar (offline):")
    print(df["customer_value_total_ever_offline"].sum())

# En fazla kazancı veya siparişi getirenleri sıralar.

In [21]:
def top_customers(df, by, top_n=10):
    """
    En fazla kazancı veya siparişi getiren müşterileri sıralar.
    """
    top_customers = df.sort_values(by=by, ascending=False).head(top_n)
    print(f"\nEn fazla {by} getiren ilk {top_n} müşteri:")
    print(top_customers[["master_id", by]])

# Veriyi hazırlar.

In [22]:
def data_preparation(df):
    """
    Veriyi hazırlar: yeni değişkenler oluşturur ve tarih sütunlarını dönüştürür.
    """
    df = create_new_variables(df)
    df = convert_date_columns(df)
    return df

# Recency, Frequency ve Monetary değerlerini hesaplar.

In [23]:
def calculate_rfm(df):
    """
    Recency, Frequency ve Monetary değerlerini hesaplar.
    """
    analysis_date = df["last_order_date"].max() + pd.DateOffset(days=2)
    df["recency"] = (analysis_date - df["last_order_date"]).dt.days
    df["frequency"] = df["total_order"]
    df["monetary"] = df["total_value"]
    rfm = df[["master_id", "recency", "frequency", "monetary"]].copy()
    return rfm

# RFM skorlarını hesaplar.

In [24]:
def calculate_rfm_scores(rfm):
    """
    RFM skorlarını hesaplar.
    """
    rfm.loc[:, "recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5, 4, 3, 2, 1])
    rfm.loc[:, "frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
    rfm.loc[:, "monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1, 2, 3, 4, 5])
    rfm.loc[:, "RF_SCORE"] = rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str)
    return rfm

# RF skorlarını segmentlere çevirir.

In [25]:
def segment_rfm(rfm):
    """
    RF skorlarını segmentlere çevirir.
    """
    seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }
    rfm.loc[:, 'segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)
    return rfm

# Belirli segmentlerdeki müşterileri kaydeder.

In [26]:
def save_segment_customers(rfm, df, segment, categories, file_name):
    """
    Belirli segmentlerdeki müşterileri kaydeder.
    """
    segment_customers = rfm[rfm["segment"].isin(segment)]
    segment_customers = segment_customers.merge(df[["master_id", "interested_in_categories_12"]], on="master_id")
    segment_customers = segment_customers[segment_customers["interested_in_categories_12"].str.contains(categories, na=False)]
    segment_customers_ids = segment_customers["master_id"]
    segment_customers_ids.to_csv(file_name, index=False)

# Tüm süreci çalıştırır.

In [27]:
def main(file_path):
    """
    Tüm süreci çalıştırır.
    """
    df = load_data(file_path)
    inspect_data(df)
    df = data_preparation(df)
    analyze_channels(df)
    top_customers(df, by="total_value")
    top_customers(df, by="total_order")
    rfm = calculate_rfm(df)
    rfm = calculate_rfm_scores(rfm)
    rfm = segment_rfm(rfm)
    
    print("\nSegmentlerin recency, frequency ve monetary ortalamaları:")
    print(rfm.groupby("segment").agg({
        "recency": "mean",
        "frequency": "mean",
        "monetary": ["mean", "count"]
    }))
    
    save_segment_customers(rfm, df, segment=["champions", "loyal_customers"], categories="KADIN", file_name="loyal_customers_ids.csv")
    save_segment_customers(rfm, df, segment=["hibernating", "at_risk", "new_customers", "about_to_sleep"], categories="ERKEK|COCUK", file_name="target_customers_ids.csv")

# Ana fonksiyonu çalıştırır.

In [28]:
file_path = 'flo_data_20k.csv'
main(file_path)

İlk 10 gözlem:
                              master_id order_channel last_order_channel  \
0  cc294636-19f0-11eb-8d74-000d3a38a36f   Android App            Offline   
1  f431bd5a-ab7b-11e9-a2fc-000d3a38a36f   Android App             Mobile   
2  69b69676-1a40-11ea-941b-000d3a38a36f   Android App        Android App   
3  1854e56c-491f-11eb-806e-000d3a38a36f   Android App        Android App   
4  d6ea1074-f1f5-11e9-9346-000d3a38a36f       Desktop            Desktop   
5  e585280e-aae1-11e9-a2fc-000d3a38a36f       Desktop            Offline   
6  c445e4ee-6242-11ea-9d1a-000d3a38a36f   Android App        Android App   
7  3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f        Mobile            Offline   
8  cfbda69e-5b4f-11ea-aca7-000d3a38a36f   Android App        Android App   
9  1143f032-440d-11ea-8b43-000d3a38a36f        Mobile             Mobile   

  first_order_date last_order_date last_order_date_online  \
0       2020-10-30      2021-02-26             2021-02-21   
1       2017-02-08      20